In [1]:
#Importar librerías
import pandas as pd
import json
import ast
import re
from textblob import TextBlob

In [2]:
#Convertir el archivo .json a dataframe
def conversion():
    with open('C:/Users/Mauro/Desktop/HENRY/Proyecto_individual/PI_1_MLOps/PI_MLOps-STEAM/user_reviews.json/australian_user_reviews.json', 'rt', encoding='utf-8') as file:
        return[ast.literal_eval(line.strip())
            for line in file]

data = conversion()

df_user_reviews = pd.DataFrame(data)

df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
# Crear una instancia del analizador de sentimiento
def sentiment_analysis(review) :
    if isinstance(review, list) and len(review) > 0:
        text = review[0].get('review', '')
        sentiment= TextBlob(text).sentiment.polarity

        if sentiment < -0.2:
            return 0 #NEGATIVO
        elif sentiment >= -0.2 and sentiment <= -0.2:
            return 1 #NEUTRAL
        else:
            return 2 #POSITIVO
    else: 
        return 1 #SIN RESEÑA

# Aplicar la función a la columna 'reviews' y crear la nueva columna 'sentiment_analysis'     
df_user_reviews['sentiment_analysis'] = df_user_reviews['reviews'].apply(sentiment_analysis)


In [4]:
# Desanidar la columna 'Reviews'

data_review = []
for index, fila in df_user_reviews.iterrows():
    for review in fila['reviews']:
        if isinstance(review, dict):
            data_review.append({
                "user_id": fila["user_id"],
                "user_url": fila["user_url"],
                "sentiment_analysis": fila["sentiment_analysis"],
                "funny": review.get("funny", ""),
                "posted": review.get("posted", ""),
                "last_edited": review.get("last_edited", ""),  
                "item_id": review.get("item_id", ""),
                "helpful": review.get("helpful", ""),
                "recommend": review.get("recommend", ""),
                "review": review.get("review", "")
            })

# Guardar el conjunto de datos en un archivo CSV
df_user_review_sentiment_analysis= pd.DataFrame(data_review)
df_user_review_sentiment_analysis.head(5)

,user_id,user_url,sentiment_analysis,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,2,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,2,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [5]:
# Eliminar columnas irrelevantes
df_user_review_sentiment_analysis.drop(['funny','last_edited','helpful','review'], axis=1, inplace=True)

print(df_user_review_sentiment_analysis.columns)



Index(['user_id', 'user_url', 'sentiment_analysis', 'posted', 'item_id',
       'recommend'],
      dtype='object')


In [6]:
#Eliminar los signos en 'posted'
df_user_review_sentiment_analysis['posted'] = df_user_review_sentiment_analysis['posted'].str.replace('.', '')
df_user_review_sentiment_analysis['posted'] = df_user_review_sentiment_analysis['posted'].str.replace(',', '')
df_user_review_sentiment_analysis['posted'] = df_user_review_sentiment_analysis['posted'].str.replace('Posted', '')

#Cambiar formato de las fechas en 'posted'
def analizar_fechas(df, columna):
    for index, fecha in enumerate(df[columna]):
        if not re.match(r'\d{2}-\d{2}-\d{4}', fecha):  #(dd-mm-AAAA)
            try:
                fecha_obj = pd.to_datetime(fecha, errors='raise')
                fecha_transformada = fecha_obj.strftime('%d-%m-%Y')
                df.at[index, columna] = fecha_transformada
            except ValueError:
               pass

analizar_fechas(df_user_review_sentiment_analysis, 'posted')

print(df_user_review_sentiment_analysis['posted'])

#Convertir 'posted' en objetos de fecha
df_user_review_sentiment_analysis['posted'] = pd.to_datetime(df_user_review_sentiment_analysis['posted'], format='mixed', dayfirst=True, errors='coerce')

#Convertir 'posted' a formato string
df_user_review_sentiment_analysis['posted'] = df_user_review_sentiment_analysis['posted'].astype(str)

#Corroborar que todos tengan el formato (dd-mm-AAAA)
formato_esperado = r'\d{2}-\d{2}-\d{4}'


0        05-11-2011
1        15-07-2011
2        21-04-2011
3        24-06-2014
4        08-09-2013
            ...    
59300       July 10
59301        July 8
59302        July 3
59303       July 20
59304        July 2
Name: posted, Length: 59305, dtype: object


In [7]:
df_user_review_sentiment_analysis.head(5)

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-11-05,1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-07-15,22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-04-21,43110,True
3,js41637,http://steamcommunity.com/id/js41637,2,2014-06-24,251610,True
4,js41637,http://steamcommunity.com/id/js41637,2,2013-09-08,227300,True


In [8]:
df_user_review_sentiment_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   user_url            59305 non-null  object
 2   sentiment_analysis  59305 non-null  int64 
 3   posted              59305 non-null  object
 4   item_id             59305 non-null  object
 5   recommend           59305 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 2.3+ MB


In [9]:
#Eliminar datos nulos 
df_user_review_sentiment_analysis=df_user_review_sentiment_analysis.dropna(subset=['posted'])
df_user_review_sentiment_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   user_url            59305 non-null  object
 2   sentiment_analysis  59305 non-null  int64 
 3   posted              59305 non-null  object
 4   item_id             59305 non-null  object
 5   recommend           59305 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 2.3+ MB


In [10]:
#Transformar columna 'recommend'
df_user_review_sentiment_analysis['recommend'] = df_user_review_sentiment_analysis['recommend'].replace({True:1, False:0})

In [11]:
#Eliminar duplicados
df_user_review_sentiment_analysis=df_user_review_sentiment_analysis.drop_duplicates(subset=['user_id'])

In [12]:
# Guardar el conjunto de datos en un archivo CSV
df_user_review_sentiment_analysis.to_csv('australian_user_reviews_sentiment_analysis.csv', index=False)

In [13]:
df_user_review_sentiment_analysis.to_parquet('australian_user_reviews_sentiment_analysis.parquet', index=False)